# Pozyskanie danych

W tym notatniku wytrenujemy, ocenimy i udostępnimy model predykcji popytu.

Wykorzystamy w tym celu podzbiór danych o sprzedaży alkoholu w stanie Iowa oraz bibliotekę autogluon.

In [36]:
import pandas as pd
# URL to the raw CSV file
url = "https://raw.githubusercontent.com/wodecki/BigDataDMBA/main/iowa_sales.csv"

# Read the CSV file directly from the URL
df = pd.read_csv(url)

# Display the first few rows of the DataFrame
print(df.tail())

            date             item_name  total_amount_sold
3599  2024-08-16  TITOS HANDMADE VODKA               4490
3600  2024-02-28  TITOS HANDMADE VODKA               8093
3601  2023-07-25  TITOS HANDMADE VODKA               4959
3602  2023-11-22  TITOS HANDMADE VODKA               7338
3603  2023-06-26  TITOS HANDMADE VODKA               8883


# Przygotowanie danych do modelowania

Wykorzystywana przez na biblioteka autogluon ma specyficzne wymagania jeśli chodzi o dane wejściowe. Poniższy skrypt transformuje nasze dane do właśnie takiego formatu.

In [37]:
# Install autogluon (AutoML library).
! uv pip -q install autogluon-timeseries

In [38]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

#transform the date column to timestamp
df['date'] = pd.to_datetime(df['date'])

# Set 'date' as the index of the DataFrame
df.set_index('date', inplace=True)
df.sort_index(inplace=True)

# Prepare the train and test datasets
# First, let's determine the exact dates we have at the end of the dataset
all_dates = df.index.unique().sort_values()  # Get all unique dates sorted
last_7_dates = all_dates[-7:]  # Take the last 7 unique dates

# Now set up the train and test datasets accordingly
test_df = df.reset_index() #train_df + extra 7 days
train_df = df[~df.index.isin(last_7_dates)].reset_index()

train_data = TimeSeriesDataFrame.from_data_frame(
    train_df,
    id_column="item_name",
    timestamp_column="date"
)

test_data = TimeSeriesDataFrame.from_data_frame(
    test_df,
    id_column="item_name",
    timestamp_column="date"
)


# Trening modelu

Mając przygotowane dane, możemy przystąpić do treningu modelu. Wykorzystamy w tym celu bibliotekę autogluon.

UWAGA: w treningu poniżej wykorzystujemy najprostsze dostępne opcje. Wyniki można istotnie poprawić wykorzystując np. cechy dynamiczne i statyczne - zainteresowanych zachęcam do lektury dokumentacji autogluon.

In [ ]:
#Train the model

predictor = TimeSeriesPredictor(
    freq="D",
    prediction_length=7,
    path="autogluon-iowa-daily",
    target="total_amount_sold",
    eval_metric="MASE",
)

predictor.fit(
    train_data,
    presets="medium_quality",
    time_limit=200,
)


Beginning AutoGluon training... Time limit = 200s
AutoGluon will save models to '/content/autogluon-iowa-daily'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
GPU Count:          0
Memory Avail:       10.14 GB / 12.67 GB (80.0%)
Disk Space Avail:   66.47 GB / 107.72 GB (61.7%)
Setting presets to: medium_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MASE,
 'freq': 'D',
 'hyperparameters': 'light',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 7,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'total_amount_sold',
 'time_limit': 200,
 'verbosity': 2}

train_data with frequency 'IRREG' has been re

# Predykcje i ich porównanie z rzeczywistością

Mamy już wytrenowany model - czas poprosić go o prognozy i porównać je z rzeczywistością.

In [ ]:
# Make predictions
predictions = predictor.predict(train_data)
predictions.head()


In [ ]:
# Plot the predictions
import matplotlib.pyplot as plt
# Plot 4 randomly chosen time series and the respective forecasts
predictor.plot(test_data, predictions, quantile_levels=[0.1, 0.9], max_history_length=200, max_num_item_ids=4)

# Symulacja wczytania wytrenowanego modelu na innej maszynie

Zobaczmy teraz, w jaki sposób możliwe jest uruchomienie modelu predykcyjnego na innym komputerze.

Model jest plikiem - wczytamy go z folderu "autogluon-iowa-daily".

Do predykcji użyjemy też naszych danych treningowych.
UWAGA: w przypadku predykcji to często stosowana praktyka. Wprowadzamy dane z przeszłości po to, by otrzymać predykcję na przyszłość.

W przykładzie poniżej znajdziesz zapytanie o "BLACK VELVET". Możesz to zmienić pytając o inne alkohole.

In [ ]:
# Now, imagine you want to load a trained model on another machine, and make some predictions...
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

# URL to the raw CSV file
url = "https://raw.githubusercontent.com/wodecki/BigDataDMBA/main/iowa_sales.csv"

# Read the CSV file directly from the URL
df = pd.read_csv(url)

#transform the date column to timestamp
df['date'] = pd.to_datetime(df['date'])

train_data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="item_name",
    timestamp_column="date"
)

predictor = TimeSeriesPredictor.load("autogluon-iowa-daily")
predictions = predictor.predict(train_data)

#get "mean" for the max value of timestamp in df for item_id = "BLACK VELVET"

# Filter the predictions for 'BLACK VELVET'
black_velvet_predictions = predictions.loc['BLACK VELVET']

# Find the maximum timestamp for 'BLACK VELVET'
max_timestamp = black_velvet_predictions.index.max()
# Get the 'mean' value for the maximum timestamp
mean_value = black_velvet_predictions.loc[max_timestamp, 'mean']
print("Mean prediction for BLACK VELVET on the maximum timestamp (", max_timestamp, "):", mean_value)


In [ ]:
# List all the alcohols

In [ ]:
df["item_name"].value_counts()

In [ ]:
# make predictions for another one
product = "TITOS HANDMADE VODKA"
product_predictions = predictions.loc[product]

# Find the maximum timestamp for product
max_timestamp = product_predictions.index.max()
# Get the 'mean' value for the maximum timestamp
mean_value = product_predictions.loc[max_timestamp, 'mean']
print("Mean prediction for Your product on (", max_timestamp, "):", mean_value)

# Uruchomienie

## Udostępnienie jako punktu dostępowego FastAPI

Czas udostępnić nasz model predykcyjny. Zaczniemy od stworzenia punktu dostępowego REST Api. Wykorzystamy do tego serwer uvicorn i bibliotekę FastAPI.

Efektem będzie adres www, na który będziemy mogli wysłać zapytanie o predykcję sprzedaży wybranego towaru na przyszły tydzień.

Zarówno zapytanie, jak i odpowiedź będą w formie tekstowej. Dla użytkownika końcowego jest to niewygodne, ale jest podstawą dla tworzenia kolejnych warstw dostępu, np. serwisów www czy aplikacji mobilnych.

In [ ]:
! pip install -q fastapi uvicorn
#!cp BigDataDMBA/main_colab.py ./
!wget https://raw.githubusercontent.com/wodecki/BigDataDMBA/main/main_colab.py -O main_colab.py
!npm install -g localtunnel
!nohup uvicorn main_colab:app --reload --host 0.0.0.0 --port 8000 &
!sleep 5  # Wait a bit for the server to start
get_ipython().system_raw('lt --port 8000 > url.txt 2>&1 &')
!sleep 5  # Additional wait time
url = !grep -o 'https://[^ ]*' url.txt
!export API_URL="{url[0]}/predict/"

In [ ]:
import urllib.parse

endpoint = url[0]
item_name = "BLACK VELVET"
# Ensure the item_name is properly URL encoded
item_name_encoded = urllib.parse.quote(item_name)
command = f"curl '{endpoint}/predict/{item_name_encoded}'"
get_ipython().system(command)

## Udostępnienie jako serwisu www

Udostępnijmy nasz serwis predykcyjny w formie aplikacji www.

Po uruchomieniu skryptów poniżej, uruchomiona zostanie strona www będąca po prostu przyjaznym dla użytkownika interfejsem dostępu do punktu końcowego API. Po wyborze interesującego nas produktu zobaczymy tabelę z prognozami sprzedaży na kolejny tydzień.

In [ ]:
! pip install -q streamlit

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

**Ważne: skopiuj nr IP powyżej do pamięci. Wkleisz go potem jako token na stronie www, która wyświetli się po wykonaniu kolejnej komendy.**

In [ ]:
import os
os.environ['API_URL'] = f"{url[0]}/predict/"
#!cp BigDataDMBA/app_colab.py ./
!wget https://raw.githubusercontent.com/wodecki/BigDataDMBA/main/app_colab.py -O app_colab.py

!nohup streamlit run app_colab.py --server.port 8501 --server.headless true --server.enableCORS false --server.enableXsrfProtection false --server.address 0.0.0.0 &

# Wait for the server to start
!sleep 5

# Use localtunnel to expose the port
get_ipython().system_raw('lt --port 8501 > url.txt 2>&1 &')

# Wait for localtunnel to establish
!sleep 5

# Get the URL
!grep -o 'https://[^ ]*' url.txt

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

**Ważne: skopiuj nr IP powyżej do pamięci. Wkleisz go potem jako token na stronie www, która wyświetli się po wykonaniu kolejnej komendy.**

In [ ]:
# Start Locust with the API URL and the web interface on port 8089
get_ipython().system_raw(
    'locust -f locustfile.py --host=$API_URL --web-port 8089 &'
)

# Wait a few seconds to ensure Locust starts
!sleep 10

# Start localtunnel to expose the web interface
get_ipython().system_raw('lt --port 8089 > url.txt 2>&1 &')

# Wait for localtunnel to establish
!sleep 5

# Display the URL for accessing the Locust web UI
!grep -o 'https://[^ ]*' url.txt